# Try out snotel_tools

In [ ]:
!mamba env create -q -f environment.yml --force
!mamba activate snotel
# then switch kernels on top right

In [ ]:
import sys
sys.path.append('snotel_tools')
import snotel_tools
import numpy as np
import shapely
import geopandas as gpd
import pandas as pd
import ulmo
import fsspec
import datetime
import mgrs
import os
import tqdm

wsdl_url = 'https://hydroportal.cuahsi.org/Snotel/cuahsi_1_1.asmx?WSDL'

In [ ]:
sites_gdf = snotel_tools.all_snotel_sites(from_ulmo=False)

In [ ]:
# try buffering
#sites_gdf['geometry'] = sites_gdf.to_crs(32611).geometry.buffer(500).to_crs(4326)

In [ ]:
sites_gdf.astype(dict(beginDate=str, endDate=str)).explore(column='mountainRange')

In [ ]:
len(np.unique(sites_gdf[sites_gdf.isActive].mgrs)) # make test bed netcdfs over just these sites

In [ ]:
aoi = shapely.geometry.Point(-120, 48)
aoi = gpd.read_file('~/sar_snow_melt_timing/input/shapefiles/olympics.geojson')

In [ ]:
sites_gdf_sorted = snotel_tools.sort_closest_snotel_sites(aoi,print_closest=True)

In [ ]:
sites_gdf_sorted.astype(dict(beginDate=str, endDate=str)).explore(column='distance_km')

In [ ]:
sites_gdf

In [ ]:
def get_variables(sitecode,print_vars=False):
    
    variables = ulmo.cuahsi.wof.get_site_info(wsdl_url, 'SNOTEL:'+sitecode)['series']
    
    if print_vars == True:
        for key,val in variables:
            print(key,val['variable']['name'])
    
    return variables

In [ ]:
v = get_variables(sites_gdf.index.values[0],print_vars=False)

In [ ]:
v.keys()
daily_vars = [x for x in v.keys() if x.endswith('_D')]
#for key,val in v.keys():
#    print(key, val['variable']['name'])

In [ ]:
daily_vars

In [ ]:
#v.keys()

In [ ]:
#v

In [ ]:
snotel_tools.download_snotel_data_csv(sites_gdf)